In [1]:
import requests
from bs4 import BeautifulSoup

#url = "https://www.capitalgroup.com/advisor/ca/en/insights/categories/markets-economy.html"
#url = "https://www.capitalgroup.com/advisor/ca/en/insights/categories/fixed-income.html"
url = "https://www.capitalgroup.com/advisor/ca/en/insights/categories/equity.html"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

# Find the list of articles
article_list = soup.find_all('li', class_='cmp-articlelist__li cmp-articlelist__non-lazyLoad')



url_list = []
article_category_list = []
# Extract article information
for article in article_list:
    title = article.find('p', class_='cmp-articlelist__title').text.strip()
    description = article.find('div', class_='cmp-articlelist__descriptiontitle').text.strip()

    # Try to extract the article URL
    try:
        article_url = "https://www.capitalgroup.com" + article.find('a', class_='cmp-articlelist__link')['href']
        url_list.append(article_url)
        article_category_list.append(title)
    except (KeyError, TypeError):
        # Handle cases where direct 'href' attribute is not available
        article_url = "URL not found"

    # Print or store the information as needed
    print(f"Title: {title}\nDescription: {description}\nURL: {article_url}\n")


Title: Global
Description: Global investing: Beyond the obvious
URL: URL not found

Title: Health Care
Description: Weight loss drugs could reshape industries beyond health care
URL: https://www.capitalgroup.com/advisor/ca/en/insights/articles/weight-loss-drugs-could-reshape-industries-beyond-health-care.html

Title: Automotive
Description: Electric vehicles in the fast lane: Who wins this race?
URL: https://www.capitalgroup.com/advisor/ca/en/insights/articles/ev-who-wins-race.html

Title: Emerging Markets
Description: Emerging markets: Positive signs even as China slows
URL: https://www.capitalgroup.com/advisor/ca/en/insights/articles/emerging-markets-positive-signs-as-china-slows.html

Title: Global Equities
Description: How I survived 23 market shocks in 35 years of investing
URL: https://www.capitalgroup.com/advisor/ca/en/insights/articles/how-survived-23-market-shocks-35-years-investing.html

Title: Global Equities
Description: Global investing: Many paths to capital appreciation


In [2]:
def extract_content(body):

    article_content = ''
    
    try:
        # Assuming you've already fetched and parsed the HTML content into the soup variable
        article_content_elements = body.find_all('div', class_='cmp-contentfragment--insights__articlefragment')

        # Extract the text content of each article fragment
        article_content = '\n'.join([element.get_text(strip=True) for element in article_content_elements])

    except Exception as err:
        print(err)
        
    return article_content

In [3]:
def extract_title(body):
    title=''
    try:
        title = body.find("title").text
    except Exception as err:
        print(err)
        
    return title
        
extract_title(soup)

'Equity | Capital Group'

In [4]:
def extract_date(body):
    
    date_info = ''
    
    try:
        # Find the meta tag with the specified property attribute
        meta_tag = body.find('meta', {'property': 'article:published_time'})
        # Extract the content attribute, which contains the date information
        date_info = meta_tag.get('content', None)
    except Exception as err:
        print(err)
        
    return date_info

In [5]:
def scrap_capital_article(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    
    attributes = dict()
    #
    title = extract_title(soup)
    
    #
    date = extract_date(soup)
    
    content = extract_content(soup)
    
    attributes["title"] = title
    attributes['date'] = date
    attributes['content'] = content
    
    
    return attributes

In [6]:
articles_info_list = []

for url in url_list:
    article_info = scrap_capital_article(url)
    articles_info_list.append(article_info)

In [7]:
import pandas as pd
df = pd.DataFrame(articles_info_list)

In [8]:
df['category']=article_category_list
df

,title,date,content,category
0,Weight loss drugs could reshape industries bey...,2023-11-22T09:52:00,"From deep-fried turkey to pumpkin pie, the hol...",Health Care
1,Electric vehicles in the fast lane: Who wins t...,2023-11-14T12:49:00,Walking the showroom floor of Germany's intern...,Automotive
2,Emerging markets: Positive signs even as China...,2023-10-18T09:27:00,The post COVID-19 era has seen a reset in fina...,Emerging Markets
3,How I survived 23 market shocks in 35 years of...,2023-09-27T01:16:00,This past winter I celebrated my 35th annivers...,Global Equities
4,Global investing: Many paths to capital apprec...,2023-09-21T09:14:00,For advisor use only. Not for use with investo...,Global Equities
...,...,...,...,...
66,Why multinationals can survive trade conflicts...,2020-02-13T02:48:00,As a portfolio manager who invests in many lar...,Trade
67,Defensive investing has paid dividends in down...,2019-04-30T03:08:00,"The Cold War was raging, Jimmy Carter became t...",Dividends
68,Keeping investors “on the train” during volati...,2018-12-10T09:18:00,Portfolio managers Alan Wilson and Hilda Applb...,Long-Term Investing
69,Investing With Common Sense and Courage | Capi...,2016-12-08T05:33:00,2016 Global Investing Forum Highlights: Portfo...,Active Management


In [9]:
df.title = df.title.map(lambda x: x.replace(" | Capital Group",""))

In [10]:
# df.to_csv("market_economy_capitalincome.csv",index = None)
#df.to_csv("fixed_income_capitalincome.csv",index = None)
#df.to_csv("equity_capitalincome.csv",index = None)

In [12]:
market_economy = pd.read_csv("market_economy_capitalincome.csv")
fixed_income = pd.read_csv("fixed_income_capitalincome.csv")
equity = pd.read_csv("equity_capitalincome.csv")

In [13]:
combined_df = pd.concat([market_economy, fixed_income, equity], ignore_index=True)

In [14]:
combined_df

,title,date,content,category
0,The U.S. avoided recession last year. What com...,2024-01-04T04:23:00,"Entering 2023, more than 85% of economists exp...",Economic Indicators
1,Economic outlook: A mixed picture for global g...,2023-11-29T10:28:00,Under the weight of elevated inflation and hig...,Economic Indicators
2,What I’m reading: American Prometheus,2023-11-09T12:11:00,"In Greek mythology, Prometheus was the god of ...",Demographics & Culture
3,2023 market trends in 5 charts,2023-10-04T10:40:00,"This year has been full of surprises, promptin...",Asset Allocation
4,"As rate hikes near end, historic investor oppo...",2023-09-14T12:00:00,To say this has been an interesting year in fi...,Asset Allocation
...,...,...,...,...
165,Why multinationals can survive trade conflicts,2020-02-13T02:48:00,As a portfolio manager who invests in many lar...,Trade
166,Defensive investing has paid dividends in down...,2019-04-30T03:08:00,"The Cold War was raging, Jimmy Carter became t...",Dividends
167,Keeping investors “on the train” during volati...,2018-12-10T09:18:00,Portfolio managers Alan Wilson and Hilda Applb...,Long-Term Investing
168,Investing With Common Sense and Courage,2016-12-08T05:33:00,2016 Global Investing Forum Highlights: Portfo...,Active Management


In [15]:
combined_df.to_csv("capital_income_combined",index =None)